#Image classification models covered :

model = EfficientNet.from_pretrained('efficientnet-b0')

model = EfficientNet.from_pretrained('efficientnet-b1')

model = EfficientNet.from_pretrained('efficientnet-b2')

model = EfficientNet.from_pretrained('efficientnet-b3')

model = EfficientNet.from_pretrained('efficientnet-b4')

model = EfficientNet.from_pretrained('efficientnet-b5')

model = EfficientNet.from_pretrained('efficientnet-b6')

model = EfficientNet.from_pretrained('efficientnet-b7')

In [1]:
import io
import os
import hashlib
import tempfile
import requests
import numpy as np
from PIL import Image
from argparse import ArgumentParser

import torch
from efficientnet_pytorch import EfficientNet
import torchvision.transforms as transforms

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

# for example we are using the Resnet18
from torchvision import models

In [2]:
# define helper functions
def fetch(url):
  # efficient loading of URLS
  fp = os.path.join(tempfile.gettempdir(), hashlib.md5(url.encode('utf-8')).hexdigest())
  if os.path.isfile(fp) and os.stat(fp).st_size > 0:
    with open(fp, "rb") as f:
      dat = f.read()
  else:
    print("fetching", url)
    dat = requests.get(url).content
    with open(fp+".tmp", "wb") as f:
      f.write(dat)
    os.rename(fp+".tmp", fp)
  return dat

def get_image_net_labels():
  with open("labels.txt", 'r') as f:
    labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip()[10:] for x in f]
  return labels_map

In [3]:
labels = get_image_net_labels()

Uncomment required model for usage

In [4]:
#model = EfficientNet.from_pretrained('efficientnet-b0')
#model = EfficientNet.from_pretrained('efficientnet-b1')
#model = EfficientNet.from_pretrained('efficientnet-b2')
#model = EfficientNet.from_pretrained('efficientnet-b3')
#model = EfficientNet.from_pretrained('efficientnet-b4')
#model = EfficientNet.from_pretrained('efficientnet-b5')
#model = EfficientNet.from_pretrained('efficientnet-b6')
#model = EfficientNet.from_pretrained('efficientnet-b7')

In [13]:
# load the model by downloading pretrained weights
model = EfficientNet.from_pretrained('efficientnet-b0')
_ = model.eval()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [14]:
# define the image transformations to apply to each image
image_transformations = transforms.Compose([
    transforms.Resize(256),                               # resize to a (256,256) image
    transforms.CenterCrop(224),                           # crop centre part of image to get (244, 244) grid
    transforms.ToTensor(),                                # convert to tensor
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),    # normalise image according to imagenet valuess
])

In [15]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/07._Camel_Profile%2C_near_Silverton%2C_NSW%2C_07.07.2007.jpg/1200px-07._Camel_Profile%2C_near_Silverton%2C_NSW%2C_07.07.2007.jpg"#"some/url/to/some/image.jpg"
image = Image.open(io.BytesIO(fetch(url)))  # load any image
x = image_transformations(image) # [3, H, W]
x = x.view(1, *x.size())
x.size()

torch.Size([1, 3, 224, 224])

In [16]:
with torch.no_grad():
    out = model(x)

In [17]:
# keep the top 10 scores
number_top = 10
probs, indices = torch.topk(out, number_top)
print(f"Top {number_top} results:")
print("===============")
for p, i in zip(probs[0], indices[0]):
    print(p, "--", labels[i])

Top 10 results:
tensor(10.6979) -- Arabian_camel,_dromedary,_Camelus_dromedarius
tensor(5.5358) -- llama
tensor(4.0592) -- gazelle
tensor(3.9918) -- bighorn,_bighorn_sheep,_cimarron,_Rocky_Mountain_bighorn,_Rocky_Mountain_sheep,_Ovis_canadensis
tensor(3.3354) -- sorrel
tensor(3.2294) -- impala,_Aepyceros_melampus
tensor(2.4651) -- ram,_tup
tensor(2.3280) -- ostrich,_Struthio_camelus
tensor(2.3230) -- ibex,_Capra_ibex
tensor(2.2993) -- koala,_koala_bear,_kangaroo_bear,_native_bear,_Phascolarctos_cinereus


Since EffcientNet uses Swish activation function, conversion to ONNX is not directly possible, hence we have to set memory efficient to False first

In [19]:
model.set_swish(memory_efficient=False)

In [21]:
# convert to ONNX
print("-"*70)
print("Now we are converting this pytorch model to ONNX model")
torch.onnx._export(model, x, f'efficientnet-b3.onnx', export_params=True)
print(f"See in the folder we have a efficientnet-b3.onnx file")
print("-"*70)


----------------------------------------------------------------------
Now we are converting this pytorch model to ONNX model
See in the folder we have a efficientnet-b3.onnx file
----------------------------------------------------------------------


ONNX to OpenVino¶

In [ ]:
from openvino.inference_engine import IECore

In [ ]:
model_xml = "fp32/efficientnet-b3.xml"
model_bin = os.path.splitext(model_xml)[0] + ".bin"
model_xml, model_bin

In [ ]:
# Plugin initialization for specified device and load extensions library if specified.
print("Creating Inference Engine...")
ie = IECore()

# Read IR
print("Loading network")
net = ie.read_network(args.model, os.path.splitext(args.model)[0] + ".bin")

print("Loading IR to the plugin...")
exec_net = ie.load_network(network=net, device_name="CPU", num_requests=2)
print(f"exec_net: {exec_net}")
print("-"*70)

In [ ]:
# convert to numpy to pass this to the IECore
x = x.numpy()

In [ ]:
# this is a bit tricky. So the input to the model is the input from ONNX graph
# IECore makes a networkX graph of the "computation graph" and when we run .infer
# it passes it through. If you are unsure of what to pass you can always check the
# <model>.xml file. In case of pytorch models the value "input.1" is the usual
# suspect. Happy Hunting!
out = exec_net.infer(inputs={"input.1": x})

# the output looks like this {"node_id": array()} so we simply load the output
out = list(out.values())[0]
print("Output Shape:", out.shape)
print("-"*70)

In [ ]:
# keep the top 10 scores
out = out[0]
number_top = 10
indices = np.argsort(out, -1)[::-1][:number_top]
probs = out[indices]
print(f"Top {number_top} results:")
print("===============")
for p, i in zip(probs, indices):
print(p, "--", labels[i])
print("-"*70)